In [2]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact

proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

project root: /Users/joachimpfefferkorn/repos/neurovolume


Somewhat adapted from a GPT response, let's first make sure that our registration applies to one frame.

In [16]:
bold_image = ants.image_read(bold_stim_filepath)  # Replace with your BOLD image path
t1_image = ants.image_read(t1_input_filepath)     # Replace with your T1 image path

In [ ]:
test_slice = ants.from_numpy(bold_image.numpy()[:,:,:,0], spacing=bold_image.spacing[:3])
slice_registration = ants.registration(
    fixed=t1_image,
    moving=test_slice,
    type_of_transform='Rigid'  # You can also use 'Affine' or 'SyN' for deformable registration
)

bold_slice_registered = ants.apply_transforms(
    fixed=t1_image,
    moving=test_slice,
    transformlist=slice_registration['fwdtransforms'],
    interpolator='linear'
)
explore_3D_vol(bold_slice_registered.numpy())

interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

In [ ]:
print(bold_slice_registered)
print(t1_image)



The following does not (it crashes the kernel):

````
bold_registered = ants.apply_transforms(
    fixed=t1_image,
    moving=bold_image,
    transformlist=registration['fwdtransforms'],
    interpolator='linear',
    imagetype=3
)
````

I suspect this has to do with a glitch on the image type 3 as this was also an issue yesterday. Perhaps open an issue on github

As a workaround, let's use a loop to cycle through all the 4th dimensional stuff.

Given the computational lift of registration, let's write them to a list



Also, perhaps you could have made this a dictionary `img:transforms`

In [ ]:
registrations = []
for frame in range(bold_image.shape[3]):
    print(f"frame {frame}/{bold_image.shape[3]}")
    print(" creating bold frame")
    bold_frame = ants.from_numpy(bold_image.numpy()[:,:,:,frame])

    print(" Registering bold frame to T1 image")
    registration = ants.registration(
    fixed=t1_image,
    moving=bold_frame,
    type_of_transform='Rigid'  # You can also use 'Affine' or 'SyN' for deformable registration
    )
    registrations += registration #This is incorrect! It's just adding the var names

My cat turned off my laptop and I lost the first 43 minute run of the transforms creation. Let's save this to a cache real quick so that doesn't happen again. (if that will even work, it looks like it's just a list of variables, that might get flushed by Python? Dig into this more)

In [71]:
with open("/Users/joachimpfefferkorn/repos/neurovolume/cache/transforms.txt", "w") as transforms_file:
    transforms_file.write(str(registrations))

First we need to create an empty 4D volume to put all these frames in

In [17]:
result_np = np.empty((t1_image.shape[0], t1_image.shape[1], t1_image.shape[2], bold_image.shape[3]))

Before we populate it, a quick test

In [27]:
print(result_np.shape)
print(bold_image.shape)
print(len(registrations))
print(registrations[0])

(512, 512, 296, 185)
(64, 64, 35, 185)
185
/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmp8mxi9mr_0GenericAffine.mat


In [54]:
print(bold_image.spacing)

(4.0, 4.0, 4.0, 2.0)


In [55]:
test_slice = ants.from_numpy(bold_image.numpy()[:,:,:,0], spacing=bold_image.spacing[:3])